##### This notebook has final code of SSLAM

In [1]:
cd C:\Users\Alfiya\Desktop\SLAB-main\Code

C:\Users\Alfiya\Desktop\SLAB-main\Code


In [2]:
import os
#Necessary packages
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import os
import joblib 
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from keras.models import load_model

from supervised_models import logit, xgb_model, mlp
from utils import perf_metric
from VIME.vime_self import vime_self
from hexr_self import hexr_self

In [3]:
def one_hot_encode(df, class_label):
    #one-hot encode class column
    ohe = OneHotEncoder()
    #Choose either class1 or class2 to select either valence or arousal
    df_ohe = pd.DataFrame(ohe.fit_transform(df[[class_label]]).toarray())
    df = df.join(df_ohe)
    
    df.drop('class1', axis=1, inplace=True)
    df.drop('class2', axis=1, inplace=True)
    df.drop('Unnamed: 0', axis=1, inplace=True)
    df.drop('valence', axis=1, inplace=True)
    df.drop('arousal', axis=1, inplace=True)
    
    X = df.loc[:,:'emg_trap']
    y = df.iloc[:,8:]
    return X, y

In [4]:
def split_data_without_normalizing(X, y, label_data_rate, test_size=0.2):
    """
    Train-test split and train split into unlablled and labelled data without normalization
    if Test is 15% of the data
    10% of 85% = 8.5% :- Labelled dataset
    90% of 85% = 76.5% :- Unlabelled dataset
    
    if Test is 20% of the data 
    10% of 80% = 8% :- Labelled dataset
    90% of 80% = 72% :- Unlabelled dataset
    """
    x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=33) 
    
    #converting to numpy arrays
    x_train = x_train.iloc[:, :].values
    y_train = y_train.iloc[:, :].values
    x_test = x_test.iloc[:,:].values
    y_test = y_test.iloc[:,:].values
    x_train.shape, x_test.shape, y_train.shape, y_test.shape
    
    # Divide labeled and unlabeled data
    idx = np.random.permutation(len(y_train))
    
    # Label data : Unlabeled data = label_data_rate:(1-label_data_rate)
    label_idx = idx[:int(len(idx)*label_data_rate)]
    unlab_idx = idx[int(len(idx)*label_data_rate):]
    
    # Unlabeled data
    x_unlab = x_train[unlab_idx, :]
    
    # Labeled data
    x_train = x_train[label_idx, :] 
    y_train = y_train[label_idx, :]
    return x_unlab, x_train, x_test, y_train, y_test

In [5]:
def split_data_with_normalization(X, y, label_data_rate, test_size=0.2):
    """
    Train-test split and train split into unlablled and labelled data with normalization
    if Test is 15% of the data
    10% of 85% = 8.5% :- Labelled dataset
    90% of 85% = 76.5% :- Unlabelled dataset
    
    if Test is 20% of the data 
    10% of 80% = 8% :- Labelled dataset
    90% of 80% = 72% :- Unlabelled dataset
    """
    from sklearn.preprocessing import MinMaxScaler

    scaler = MinMaxScaler()
    
    X[['ecg', 'bvp', 'gsr', 'rsp', 'skt', 'emg_zygo', 'emg_coru', 'emg_trap']] = scaler.fit_transform(X[['ecg', 'bvp', 'gsr', 'rsp', 'skt', 'emg_zygo', 'emg_coru', 'emg_trap']])

    x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=33) 
    
    #converting to numpy arrays
    x_train = x_train.iloc[:, :].values
    y_train = y_train.iloc[:, :].values
    x_test = x_test.iloc[:,:].values
    y_test = y_test.iloc[:,:].values
    x_train.shape, x_test.shape, y_train.shape, y_test.shape
    
    # Divide labeled and unlabeled data
    idx = np.random.permutation(len(y_train))
    
    # Label data : Unlabeled data = label_data_rate:(1-label_data_rate)
    label_idx = idx[:int(len(idx)*label_data_rate)]
    unlab_idx = idx[int(len(idx)*label_data_rate):]
    
    # Unlabeled data
    x_unlab = x_train[unlab_idx, :]
    
    # Labeled data
    x_train = x_train[label_idx, :] 
    y_train = y_train[label_idx, :]
    return x_unlab, x_train, x_test, y_train, y_test

In [6]:
#Experimental parameters
label_data_rate = 0.1

#Metrics
metric1 = 'acc'
metric2 = 'auc'

#parameters to train the encoder
p_m = 0.3
alpha = 2.0

## SSLAM

### SSLAM Valence

In [16]:
#load case dataset
df = pd.read_csv("..\Data\CASE_2class.csv")

In [17]:
X, Y = one_hot_encode(df, 'class1')

In [89]:
x_unlab, x_train, x_test, y_train, y_test = split_data_with_normalization(X, Y, label_data_rate, test_size=0.15)

In [90]:
# Train SLAB-Self encoder
slab_self_parameters = dict()
slab_self_parameters['batch_size'] = 128
slab_self_parameters['epochs'] = 10
slab_valence_encoder = hexr_self(x_unlab, p_m, alpha, slab_self_parameters)
    
# Save encoder
if not os.path.exists('Models'):
  os.makedirs('Models')

file_name = 'Models/SLAB_valence_encoder11.h5'
    
slab_valence_encoder.save(file_name)  

Epoch 1/10
8792/8792 [==============================] - 7s 695us/step - loss: 0.6215 - mask_loss: 0.6120 - feature_loss: 0.0048
Epoch 2/10
8792/8792 [==============================] - 6s 695us/step - loss: 0.6186 - mask_loss: 0.6109 - feature_loss: 0.0039
Epoch 3/10
8792/8792 [==============================] - 6s 655us/step - loss: 0.6180 - mask_loss: 0.6103 - feature_loss: 0.0039
Epoch 4/10
8792/8792 [==============================] - 5s 600us/step - loss: 0.6167 - mask_loss: 0.6090 - feature_loss: 0.0039
Epoch 5/10
8792/8792 [==============================] - 5s 597us/step - loss: 0.6154 - mask_loss: 0.6077 - feature_loss: 0.0039
Epoch 6/10
8792/8792 [==============================] - 5s 591us/step - loss: 0.6142 - mask_loss: 0.6065 - feature_loss: 0.0038
Epoch 7/10
8792/8792 [==============================] - 5s 593us/step - loss: 0.6134 - mask_loss: 0.6057 - feature_loss: 0.0039
Epoch 8/10
8792/8792 [==============================] - 5s 593us/step - loss: 0.6127 - mask_loss: 0.6050

In [91]:
#predict and evaluate slab self encoder
slab_valence_encoder = load_model('Models/SLAB_valence_encoder11.h5')

# Test SLAB
x_train_hat = slab_valence_encoder.predict(x_train)
x_test_hat = slab_valence_encoder.predict(x_test)

slab_valence_acc = []
slab_valence_auc = []

mlp_parameters = dict()
mlp_parameters['hidden_dim'] = 100
mlp_parameters['epochs'] = 100
mlp_parameters['activation'] = 'relu'
mlp_parameters['batch_size'] = 128

6896/6896 [==============================] - 3s 394us/step


In [92]:
7

In [22]:
#experiment to compute accuracy for different number of hidden layers
mlp_parameters['num_layers'] = 5
for i in range (0, 4):
    y_test_hat = mlp(x_train_hat, y_train, x_test_hat, mlp_parameters)
    slab_valence_acc.append(perf_metric(metric1, y_test, y_test_hat))
    slab_valence_auc.append(perf_metric(metric2, y_test, y_test_hat))

6896/6896 [==============================] - 4s 584us/step
He weight initialization
6896/6896 [==============================] - 4s 594us/step
He weight initialization
6896/6896 [==============================] - 4s 583us/step
He weight initialization
6896/6896 [==============================] - 4s 585us/step
He weight initialization


In [101]:
mlp_parameters['num_layers'] = 5
y_test_hat = mlp(x_train_hat, y_train, x_test_hat, mlp_parameters)
slab_valence_acc.append(perf_metric(metric1, y_test, y_test_hat))
slab_valence_auc.append(perf_metric(metric2, y_test, y_test_hat))

KeyboardInterrupt: 

# He Uniform Initialization with normalization

In [ ]:
np.round(slab_valence_acc, 4)

In [85]:
np.round(slab_valence_auc,4)

array([0.9113, 0.8952, 0.8973, 0.902 , 0.8982, 0.888 ])

In [86]:
print("Mean of valence accuracies for SLAB using He weight initialization is ",round(np.mean(slab_valence_acc),4))

# Compute standard deviation
std_dev_acc = np.std(slab_valence_acc)
print("Standard Deviation of Valence Accuracies using SLAB: ", round(std_dev_acc,4))

print("Mean of valence accuracies AUC Score for SLAB is ",round(np.mean(slab_valence_auc),4))

Mean of valence accuracies for SLAB using He weight initialization is  0.8346
Standard Deviation of Valence Accuracies using SLAB:  0.0069
Mean of valence accuracies AUC Score for SLAB is  0.8987


# He Uniform Initialization

In [84]:
np.round(slab_valence_acc, 4)

array([0.8465, 0.8313, 0.8351, 0.8377, 0.8337, 0.8234])

In [85]:
np.round(slab_valence_auc,4)

array([0.9113, 0.8952, 0.8973, 0.902 , 0.8982, 0.888 ])

In [86]:
print("Mean of valence accuracies for SLAB using He weight initialization is ",round(np.mean(slab_valence_acc),4))

# Compute standard deviation
std_dev_acc = np.std(slab_valence_acc)
print("Standard Deviation of Valence Accuracies using SLAB: ", round(std_dev_acc,4))

print("Mean of valence accuracies AUC Score for SLAB is ",round(np.mean(slab_valence_auc),4))

Mean of valence accuracies for SLAB using He weight initialization is  0.8346
Standard Deviation of Valence Accuracies using SLAB:  0.0069
Mean of valence accuracies AUC Score for SLAB is  0.8987


### SLAB - Arousal

In [19]:
#load case dataset
df = pd.read_csv("..\Data\CASE_2class.csv")

In [20]:
X, Y = one_hot_encode(df, 'class2')

In [21]:
x_unlab, x_train, x_test, y_train, y_test = split_data_without_normalizing(X, Y, label_data_rate, test_size=0.15)

In [22]:
# Train SLAB-Self encoder
slab_self_parameters = dict()
slab_self_parameters['batch_size'] = 128
slab_self_parameters['epochs'] = 10
slab_encoder_arousal = hexr_self(x_unlab, p_m, alpha, slab_self_parameters)
    
# Save encoder
if not os.path.exists('Models'):
  os.makedirs('Models')

file_name = 'Models/SLAB_arousal_encoder4.h5'
    
slab_encoder_arousal.save(file_name)  

Epoch 1/10
8792/8792 [==============================] - 5s 583us/step - loss: 7.9170 - mask_loss: 0.6138 - feature_loss: 3.6516
Epoch 2/10
8792/8792 [==============================] - 5s 579us/step - loss: 4.8456 - mask_loss: 0.6125 - feature_loss: 2.1165
Epoch 3/10
8792/8792 [==============================] - 5s 579us/step - loss: 4.5650 - mask_loss: 0.6118 - feature_loss: 1.9766
Epoch 4/10
8792/8792 [==============================] - 5s 577us/step - loss: 4.4778 - mask_loss: 0.6123 - feature_loss: 1.9328
Epoch 5/10
8792/8792 [==============================] - 5s 581us/step - loss: 4.3919 - mask_loss: 0.6118 - feature_loss: 1.8901
Epoch 6/10
8792/8792 [==============================] - 5s 580us/step - loss: 4.3528 - mask_loss: 0.6114 - feature_loss: 1.8707
Epoch 7/10
8792/8792 [==============================] - 5s 574us/step - loss: 4.3318 - mask_loss: 0.6114 - feature_loss: 1.8602
Epoch 8/10
8792/8792 [==============================] - 5s 580us/step - loss: 4.3134 - mask_loss: 0.6114

In [23]:
#predict and evaluate slab self encoder
slab_encoder_arousal = load_model('Models/SLAB_arousal_encoder4.h5')

# Test SLAB
x_train_hat = slab_encoder_arousal.predict(x_train)
x_test_hat = slab_encoder_arousal.predict(x_test)

slab_arousal_acc = []
slab_arousal_auc = []

mlp_parameters = dict()
mlp_parameters['hidden_dim'] = 100
mlp_parameters['epochs'] = 100
mlp_parameters['activation'] = 'relu'
mlp_parameters['batch_size'] = 128

6896/6896 [==============================] - 3s 373us/step


In [24]:
#experiment to compute accuracy for different number of hidden layers
mlp_parameters['num_layers'] = 5
for i in range (0, 5):
    y_test_hat = mlp(x_train_hat, y_train, x_test_hat, mlp_parameters)
    slab_arousal_acc.append(perf_metric(metric1, y_test, y_test_hat))
    slab_arousal_auc.append(perf_metric(metric2, y_test, y_test_hat))

KeyboardInterrupt: 

In [ ]:
np.round(slab_arousal_acc,4)

In [ ]:
np.round(slab_arousal_auc, 4)

In [ ]:
print("Mean of arousal accuracies for SLAB using He weight initialization is ",round(np.mean(slab_arousal_acc),4))

print("Mean of arousal accuracies AUC Score for SLAB is ",round(np.mean(slab_arousal_auc), 4))

# Compute standard deviation
std_dev_acc = np.std(slab_arousal_acc)
print("Standard Deviation of arousal Accuracies using SLAB: ", std_dev_acc)